In [0]:
from pyspark.sql import functions as F

df = spark.read.format("delta").table("bank_silver")

print("Rows:", df.count())
print("Columns:", len(df.columns))
df.printSchema()

In [0]:
# Month to number for ordering
month_map = {"jan":1,"feb":2,"mar":3,"apr":4,"may":5,"jun":6,
             "jul":7,"aug":8,"sep":9,"oct":10,"nov":11,"dec":12}

mapping_expr = F.create_map([F.lit(x) for pair in month_map.items() for x in pair])

df = df.withColumn("month_num", mapping_expr[F.col("month")])

# Quarter from month number
df = df.withColumn("quarter",
    F.when(F.col("month_num") <= 3, "Q1")
     .when(F.col("month_num") <= 6, "Q2")
     .when(F.col("month_num") <= 9, "Q3")
     .otherwise("Q4")
)

# Verify
df.select("month", "month_num", "quarter").distinct().orderBy("month_num").show()

In [0]:
# How efficient was the contact (longer call + fewer attempts = more efficient)
df = df.withColumn("contact_efficiency",
    F.round(F.col("duration") / F.col("campaign"), 2)
)

# Campaign intensity score (how aggressively customer was called)
df = df.withColumn("campaign_intensity",
    F.when(F.col("campaign") == 1, "Low")
     .when(F.col("campaign") <= 3, "Medium")
     .otherwise("High")
)

# High value segment flag (our golden segment from EDA)
df = df.withColumn("high_value_segment",
    F.when(
        (F.col("job").isin("student", "retired")) & 
        (F.col("balance_tier").isin("Medium", "High")),
        1
    ).otherwise(0)
)

# Verify
df.select("duration", "campaign", "contact_efficiency", 
          "campaign_intensity", "high_value_segment").show(5)

In [0]:
# Cost per call assumption (total budget / total calls)
COST_PER_CALL = round(1200000 / df.count(), 2)
print(f"Assumed cost per call: €{COST_PER_CALL}")

# Total cost per customer (cost per call x number of calls)
df = df.withColumn("customer_acquisition_cost",
    F.round(F.lit(COST_PER_CALL) * F.col("campaign"), 2)
)

# Assumed term deposit revenue if converted
REVENUE_IF_CONVERTED = 2000  

# Expected revenue (probability will come from ML — for now use segment averages from EDA)
df = df.withColumn("expected_revenue",
    F.when(F.col("high_value_segment") == 1, 
           F.lit(REVENUE_IF_CONVERTED * 0.28))  # 28% conversion rate for golden segment
     .otherwise(F.lit(REVENUE_IF_CONVERTED * 0.11))  # 11% baseline
)

# ROI per customer
df = df.withColumn("estimated_roi",
    F.round((F.col("expected_revenue") - F.col("customer_acquisition_cost")) / 
             F.col("customer_acquisition_cost") * 100, 2)
)

# Verify
df.select("campaign", "customer_acquisition_cost", 
          "expected_revenue", "estimated_roi", 
          "high_value_segment").show(5)
          

In [0]:
df = spark.read.format("delta").table("bank_silver")
df.columns

In [0]:
from pyspark.sql import functions as F

df = spark.read.format("delta").table("bank_silver")

# month to number
month_map = {"jan":1,"feb":2,"mar":3,"apr":4,"may":5,"jun":6,
             "jul":7,"aug":8,"sep":9,"oct":10,"nov":11,"dec":12}

mapping_expr = F.create_map([F.lit(x) for pair in month_map.items() for x in pair])

df = df.withColumn("month_num", mapping_expr[F.col("month")])

# quarter from month number
df = df.withColumn("quarter",
    F.when(F.col("month_num") <= 3, "Q1")
     .when(F.col("month_num") <= 6, "Q2")
     .when(F.col("month_num") <= 9, "Q3")
     .otherwise("Q4")
)

# verify
df.select("month", "month_num", "quarter").distinct().orderBy("month_num").show()

In [0]:
# How aggressively was the customer called this campaign
df = df.withColumn("campaign_intensity",
    F.when(F.col("campaign") == 1, "Low")
     .when(F.col("campaign") <= 3, "Medium")
     .otherwise("High")
)

# Quality of contact — seconds per call attempt
# Higher = longer calls with fewer attempts = better quality contact
df = df.withColumn("contact_efficiency",
    F.round(F.col("duration") / F.col("campaign"), 2)
)

# Verify
df.select("campaign", "campaign_intensity", "duration", "contact_efficiency").show(5)

In [0]:

df = df.withColumn("high_value_segment",
    F.when(
        (F.col("job").isin("student", "retired")) &
        (F.col("balance_tier").isin("Medium", "High")),
        1
    ).otherwise(0)
)


df = df.withColumn("y_label",
    F.when(F.col("y") == "yes", 1).otherwise(0)
)


print("High value segment distribution:")
df.groupBy("high_value_segment").count().show()

print("Target variable distribution:")
df.groupBy("y_label", "y").count().show()

In [0]:
print("All columns now:", df.columns)
print("Total columns:", len(df.columns))

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

dim_customer = df.select(
    monotonically_increasing_id().alias("customer_id"),
    "age", "age_group",
    "job", "marital", "education",
    "default", "balance", "balance_log",
    "balance_tier", "housing", "loan",
    "high_value_segment"
)

dim_customer.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("bank_dim_customer")

print("dim_customer rows:", dim_customer.count())
dim_customer.show(3)

In [0]:
dim_campaign = df.select(
    monotonically_increasing_id().alias("campaign_id"),
    "contact", "campaign", "campaign_intensity",
    "duration", "contact_efficiency",
    "was_previously_contacted", "previous", "poutcome"
)

dim_campaign.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("bank_dim_campaign")

print("dim_campaign rows:", dim_campaign.count())
dim_campaign.show(3)

In [0]:
dim_date = df.select(
    monotonically_increasing_id().alias("date_id"),
    "day", "month", "month_num", "quarter"
).distinct()

dim_date.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("bank_dim_date")

print("dim_date rows:", dim_date.count())
dim_date.orderBy("month_num", "day").show(5)

In [0]:
# Cost assumptions
COST_PER_CALL = round(1200000 / df.count(), 2)
REVENUE_IF_CONVERTED = 2000

df_fact = df.withColumn("cost_per_call", F.lit(COST_PER_CALL)) \
            .withColumn("total_cost", 
                F.round(F.lit(COST_PER_CALL) * F.col("campaign"), 2)) \
            .withColumn("revenue_if_converted", F.lit(REVENUE_IF_CONVERTED)) \
            .withColumn("actual_revenue",
                F.when(F.col("y_label") == 1, REVENUE_IF_CONVERTED).otherwise(0))

fact_marketing = df_fact.select(
    monotonically_increasing_id().alias("fact_id"),
    # keys
    "y", "y_label",
    # campaign metrics
    "campaign", "duration", "pdays", "previous",
    "contact_efficiency", "campaign_intensity",
    "was_previously_contacted", "poutcome",
    # customer profile
    "age_group", "balance_tier", "job",
    "marital", "education", "high_value_segment",
    # time
    "day", "month", "month_num", "quarter",
    # financials
    "cost_per_call", "total_cost",
    "revenue_if_converted", "actual_revenue"
)

fact_marketing.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("bank_fact_marketing")

print("fact_marketing rows:", fact_marketing.count())
fact_marketing.show(3)

In [0]:
print("Tables saved:")
print("  dim_customer  :", spark.read.format("delta").table("bank_dim_customer").count())
print("  dim_campaign  :", spark.read.format("delta").table("bank_dim_campaign").count())
print("  dim_date      :", spark.read.format("delta").table("bank_dim_date").count())
print("  fact_marketing:", spark.read.format("delta").table("bank_fact_marketing").count())

In [0]:

df.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable("bank_gold")

print("Gold table saved:", spark.read.format("delta").table("bank_gold").count(), "rows")
print("Gold columns:", spark.read.format("delta").table("bank_gold").columns)

In [0]:
print("All tables saved:")
print("  bank_silver      :", spark.read.format("delta").table("bank_silver").count())
print("  bank_gold        :", spark.read.format("delta").table("bank_gold").count())
print("  bank_dim_customer:", spark.read.format("delta").table("bank_dim_customer").count())
print("  bank_dim_campaign:", spark.read.format("delta").table("bank_dim_campaign").count())
print("  bank_dim_date    :", spark.read.format("delta").table("bank_dim_date").count())
print("  bank_fact_marketing:", spark.read.format("delta").table("bank_fact_marketing").count())